In [8]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

## Загрузка данных

In [9]:
data = pd.read_csv('./data/amine_treatment-final.csv', on_bad_lines='skip')
data

,"feed_gas temperature, C","feed_gas pressure, kPa","feed_gas mass flow, kg/h",feed_gas molecular weight,"feed_gas Mass density, kg/m3","feed_gas actual liquid flow, m3/h","feed_gas actual vapour flow, m3/h",feed_gas CO2 mol frac,feed_gas Methane mol frac,feed_gas Ethane mol frac,...,degased_amine Propane mol frac,degased_amine i-Butane mol frac,degased_amine n-Butane mol frac,degased_amine i-Pentane mol frac,degased_amine n-Pentane mol frac,degased_amine H2S mol frac,degased_amine H2O mol frac,degased_amine MDEAmine mol frac,degased_amine H2S ppm,degased_amine CO2 ppm
0,52.0,6701.325,259647.0,24.668523,80.705482,0.000000,3217.216383,0.031875,0.633122,0.192729,...,0.000004,1.979803e-06,6.785930e-06,3.039502e-08,1.629357e-07,0.014838,0.910916,0.062270,22521.880092,23245.607009
1,40.0,6701.325,63853.0,23.411939,79.292250,0.000000,805.286774,0.039520,0.681947,0.179992,...,0.000006,1.099164e-07,7.886635e-07,2.425456e-06,2.679624e-08,0.010470,0.896612,0.077190,15891.219056,30468.484150
2,46.0,6701.325,15928.0,27.803853,108.358884,0.000000,146.993024,0.027859,0.577870,0.163095,...,0.000018,5.179434e-05,2.863001e-05,4.721545e-06,1.359401e-06,0.001284,0.860268,0.132676,1949.372531,10648.306282
3,36.0,6701.325,150276.0,22.053344,74.092459,0.000000,2028.222609,0.005420,0.856717,0.000249,...,0.000039,8.734358e-06,3.706189e-05,5.426945e-05,3.949909e-06,0.001233,0.809722,0.184359,1871.528366,8438.736973
4,12.0,6701.325,52691.0,24.334510,114.182302,0.002870,461.460940,0.027297,0.694634,0.091063,...,0.000100,7.719691e-05,2.661312e-05,1.351350e-05,6.619552e-06,0.003300,0.788340,0.200715,5008.207346,14011.150105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512795,18.0,6701.325,278813.0,27.267819,147.437736,0.239690,1890.816202,0.036621,0.666807,0.042159,...,0.000053,1.226987e-04,3.336251e-05,4.145371e-06,8.510407e-06,0.002101,0.796328,0.187149,3188.443718,27411.897443
512796,33.0,6701.325,72612.0,26.629816,112.530303,0.000000,645.266193,0.025383,0.605142,0.150361,...,0.000049,8.271254e-05,1.199104e-05,1.810930e-06,4.749955e-06,0.003784,0.826966,0.162727,5742.847844,11941.343733
512797,17.0,6701.325,272718.0,26.290335,135.390018,0.133398,2014.180541,0.039356,0.606850,0.167318,...,0.000037,2.874297e-05,3.343060e-06,1.045919e-05,2.187381e-06,0.045335,0.725301,0.179546,68812.636471,97071.672175
512798,59.0,6701.325,75970.0,24.231673,73.727565,0.000000,1030.415146,0.037951,0.732049,0.086286,...,0.000013,1.782460e-05,8.286743e-06,5.417825e-06,2.393693e-07,0.002551,0.877567,0.106280,3871.441495,26287.366187


In [10]:
list(data.columns)

['feed_gas temperature, C',
 'feed_gas pressure, kPa',
 'feed_gas mass flow, kg/h',
 'feed_gas molecular weight',
 'feed_gas Mass density, kg/m3',
 'feed_gas actual liquid flow, m3/h',
 'feed_gas actual vapour flow, m3/h',
 'feed_gas CO2 mol frac',
 'feed_gas Methane mol frac',
 'feed_gas Ethane mol frac',
 'feed_gas Propane mol frac',
 'feed_gas i-Butane mol frac',
 'feed_gas n-Butane mol frac',
 'feed_gas i-Pentane mol frac',
 'feed_gas n-Pentane mol frac',
 'feed_gas H2S mol frac',
 'feed_gas H2O mol frac',
 'feed_gas MDEAmine mol frac',
 'feed_gas H2S ppm',
 'feed_gas CO2 ppm',
 'Absorber number of stages',
 'lean_amine temperature, C',
 'lean_amine pressure, kPa',
 'lean_amine mass flow, kg/h',
 'lean_amine molecular weight',
 'lean_amine Mass density, kg/m3',
 'lean_amine actual liquid flow, m3/h',
 'lean_amine actual vapour flow, m3/h',
 'lean_amine CO2 mol frac',
 'lean_amine Methane mol frac',
 'lean_amine Ethane mol frac',
 'lean_amine Propane mol frac',
 'lean_amine i-Butane

In [11]:
data['rich_amine molar flow, kgmol/h'] = data['rich_amine mass flow, kg/h'] / data['rich_amine molecular weight']
data['rich_amine H2S molar flow, kgmol/h'] = data['rich_amine molar flow, kgmol/h'] * data['rich_amine H2S mol frac']
data['rich_amine CO2 molar flow, kgmol/h'] = data['rich_amine molar flow, kgmol/h'] * data['rich_amine CO2 mol frac']
data['rich_amine H2O molar flow, kgmol/h'] = data['rich_amine molar flow, kgmol/h'] * data['rich_amine H2O mol frac']
data['rich_amine MDEAmine molar flow, kgmol/h'] = data['rich_amine molar flow, kgmol/h'] * data['rich_amine MDEAmine mol frac']

data['feed_gas molar flow, kgmol/h'] = data['feed_gas mass flow, kg/h'] / data['feed_gas molecular weight']
data['feed_gas H2S molar flow, kgmol/h'] = data['feed_gas molar flow, kgmol/h'] * data['feed_gas H2S mol frac']
data['feed_gas CO2 molar flow, kgmol/h'] = data['feed_gas molar flow, kgmol/h'] * data['feed_gas CO2 mol frac']
data['feed_gas H2O molar flow, kgmol/h'] = data['feed_gas molar flow, kgmol/h'] * data['feed_gas H2O mol frac']
data['feed_gas MDEAmine molar flow, kgmol/h'] = data['feed_gas molar flow, kgmol/h'] * data['feed_gas MDEAmine mol frac']


data['lean_amine molar flow, kgmol/h'] = data['lean_amine mass flow, kg/h'] / data['lean_amine molecular weight']
data['lean_amine H2S molar flow, kgmol/h'] = data['lean_amine molar flow, kgmol/h'] * data['lean_amine H2S mol frac']
data['lean_amine CO2 molar flow, kgmol/h'] = data['lean_amine molar flow, kgmol/h'] * data['lean_amine CO2 mol frac']
data['lean_amine H2O molar flow, kgmol/h'] = data['lean_amine molar flow, kgmol/h'] * data['lean_amine H2O mol frac']
data['lean_amine MDEAmine molar flow, kgmol/h'] = data['lean_amine molar flow, kgmol/h'] * data['lean_amine MDEAmine mol frac']

## Модель для определения температуры продуктов

In [3]:
mini_data = data.sample(frac=0.25)
len(mini_data)

128200

In [6]:
sulphur = data[['feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac',
                'feed_gas Methane mol frac', 'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 
                'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac',
                'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
                'feed_gas MDEAmine mol frac',
                'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
                'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
                'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
                'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac',
                'lean_amine MDEAmine mol frac', 'sweet_gas temperature, C', 'rich_amine temperature, C'
               ]]

X = sulphur[[
                'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac',
                'feed_gas Methane mol frac', 'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 
                'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac',
                'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
                'feed_gas MDEAmine mol frac',
                'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
                'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
                'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
                'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac',
                'lean_amine MDEAmine mol frac'
]]

y = sulphur[['sweet_gas temperature, C', 'rich_amine temperature, C']]

# Собираем трансформер столбцов
ct = make_column_transformer(
    (MinMaxScaler(), ['feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac',
                'feed_gas Methane mol frac', 'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 
                'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac',
                'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
                'feed_gas MDEAmine mol frac',
                'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
                'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
                'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
                'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac',
                'lean_amine MDEAmine mol frac'])
)

# Создаем обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Загружаем тренировочную выборку
ct.fit(X_train)

# Нормализуем данные
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

X_train_normal.shape, y_train.shape

((410240, 26), (410240, 2))

In [86]:
# Set random set
tf.random.set_seed(42)

# 1. Create a model
prod_temp_model = tf.keras.Sequential([
    tf.keras.layers.Dense(26, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(2)
])

# 2. Compile the model
prod_temp_model.compile(
    loss='mae',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['mae']
)

# 3. Fit the model
prod_temp_model.fit(X_train_normal, y_train, epochs=20)

Epoch 1/20
12820/12820 [==============================] - 31s 2ms/step - loss: 2.8807 - mae: 2.8807
Epoch 2/20
12820/12820 [==============================] - 31s 2ms/step - loss: 1.7489 - mae: 1.7489
Epoch 3/20
12820/12820 [==============================] - 31s 2ms/step - loss: 1.5962 - mae: 1.5962
Epoch 4/20
12820/12820 [==============================] - 32s 2ms/step - loss: 1.4959 - mae: 1.4959
Epoch 5/20
12820/12820 [==============================] - 31s 2ms/step - loss: 1.4451 - mae: 1.4451
Epoch 6/20
12820/12820 [==============================] - 32s 2ms/step - loss: 1.4027 - mae: 1.4027
Epoch 7/20
12820/12820 [==============================] - 31s 2ms/step - loss: 1.3708 - mae: 1.3708
Epoch 8/20
12820/12820 [==============================] - 32s 2ms/step - loss: 1.3418 - mae: 1.3418
Epoch 9/20
12820/12820 [==============================] - 31s 2ms/step - loss: 1.3195 - mae: 1.3195
Epoch 10/20
12820/12820 [==============================] - 32s 2ms/step - loss: 1.3019 - mae: 1.3019

In [87]:
y_pred = prod_temp_model.predict(X_test_normal)

3205/3205 [==============================] - 4s 1ms/step


In [107]:
y_pred[101]

array([57.381298, 55.735657], dtype=float32)

In [108]:
y_test.iloc[101]

sweet_gas temperature, C     57.480083
rich_amine temperature, C    55.968702
Name: 237204, dtype: float64

In [110]:
100 - ((55.735657 * 100) / 55.968702)

0.4163845000371822

In [111]:
prod_temp_model.save('./models/amine_treatment/amine_treatment_prod_temp_model.h5')

In [12]:
sulphur_data = sulphur.sample(frac=0.01)
sulphur_data

,"feed_gas temperature, C","feed_gas mass flow, kg/h",feed_gas CO2 mol frac,feed_gas Methane mol frac,feed_gas Ethane mol frac,feed_gas Propane mol frac,feed_gas i-Butane mol frac,feed_gas n-Butane mol frac,feed_gas i-Pentane mol frac,feed_gas n-Pentane mol frac,...,lean_amine Propane mol frac,lean_amine i-Butane mol frac,lean_amine n-Butane mol frac,lean_amine i-Pentane mol frac,lean_amine n-Pentane mol frac,lean_amine H2S mol frac,lean_amine H2O mol frac,lean_amine MDEAmine mol frac,"sweet_gas temperature, C","rich_amine temperature, C"
229777,59.0,295333.0,0.037486,0.709195,0.027945,0.049124,0.096944,0.016101,0.022952,0.004392,...,0.0,0.0,0.0,0.0,0.0,0.000211,0.977453,0.019753,63.534447,65.537233
230248,22.0,51765.0,0.036442,0.649914,0.060758,0.072099,0.052338,0.046696,0.041313,0.003336,...,0.0,0.0,0.0,0.0,0.0,0.000344,0.810059,0.184826,25.056876,30.451538
142441,53.0,94584.0,0.026240,0.633435,0.186715,0.090658,0.019003,0.011180,0.025023,0.002668,...,0.0,0.0,0.0,0.0,0.0,0.000291,0.890271,0.108304,26.411256,36.111568
236140,39.0,135235.0,0.028697,0.800068,0.024626,0.082167,0.009194,0.023052,0.017758,0.006773,...,0.0,0.0,0.0,0.0,0.0,0.000181,0.839054,0.157503,35.072823,45.195079
328006,50.0,27226.0,0.024775,0.728327,0.096035,0.039802,0.018610,0.062545,0.017170,0.009563,...,0.0,0.0,0.0,0.0,0.0,0.000238,0.899695,0.097257,40.644903,46.896273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127774,57.0,106247.0,0.007134,0.564712,0.150123,0.092254,0.052006,0.076666,0.035214,0.003127,...,0.0,0.0,0.0,0.0,0.0,0.000139,0.860390,0.136670,33.316939,47.659560
340722,12.0,139188.0,0.014490,0.654710,0.126441,0.074161,0.058075,0.011644,0.014428,0.007704,...,0.0,0.0,0.0,0.0,0.0,0.000116,0.858768,0.137278,23.306435,25.228895
460614,20.0,278135.0,0.020406,0.661170,0.083639,0.022305,0.068197,0.078623,0.049696,0.000717,...,0.0,0.0,0.0,0.0,0.0,0.000386,0.845761,0.152531,21.611649,26.587039
437188,43.0,94367.0,0.028831,0.658443,0.177795,0.036555,0.024317,0.051922,0.007627,0.000021,...,0.0,0.0,0.0,0.0,0.0,0.000317,0.980460,0.017175,32.290935,35.745604


In [13]:
sulphur_data.to_csv('sulphur_data.csv')

## Модель для определения массовых расходов продуктовых потоков

In [92]:
mini_data = data.sample(frac=0.85)
len(mini_data)

435880

In [122]:
prod_temp = data[[
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
    'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac',
    'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
    'feed_gas MDEAmine mol frac', 'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
     'rich_amine mass flow, kg/h', 'sweet_gas mass flow, kg/h'
               ]]

X = prod_temp[[
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
    'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac',
    'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
    'feed_gas MDEAmine mol frac', 'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
]]

y = prod_temp[['rich_amine mass flow, kg/h', 'sweet_gas mass flow, kg/h']]

# Собираем трансформер столбцов
ct = make_column_transformer(
    (MinMaxScaler(), [
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
    'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac',
    'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
    'feed_gas MDEAmine mol frac', 'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
    ])
)

# Создаем обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Загружаем тренировочную выборку
ct.fit(X_train)

# Нормализуем данные
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

X_train_normal.shape, y_train.shape

((410240, 26), (410240, 2))

In [123]:
# Set random set
tf.random.set_seed(42)

# 1. Create a model
prod_mass_flow_model = tf.keras.Sequential([
    tf.keras.layers.Dense(26, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(2)
])

# 2. Compile the model
prod_mass_flow_model.compile(
    loss='mae',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['mae']
)

# 3. Fit the model
prod_mass_flow_model.fit(X_train_normal, y_train, epochs=10)

Epoch 1/10
12820/12820 [==============================] - 28s 2ms/step - loss: 50311.9336 - mae: 50311.9336
Epoch 2/10
12820/12820 [==============================] - 24s 2ms/step - loss: 43860.3086 - mae: 43860.3086
Epoch 3/10
12820/12820 [==============================] - 28s 2ms/step - loss: 43816.8516 - mae: 43816.8516
Epoch 4/10
12820/12820 [==============================] - 33s 3ms/step - loss: 30582.1582 - mae: 30582.1582
Epoch 5/10
12820/12820 [==============================] - 33s 3ms/step - loss: 1560.9634 - mae: 1560.9634
Epoch 6/10
12820/12820 [==============================] - 28s 2ms/step - loss: 1439.8798 - mae: 1439.8798
Epoch 7/10
12820/12820 [==============================] - 28s 2ms/step - loss: 1413.0997 - mae: 1413.0997
Epoch 8/10
12820/12820 [==============================] - 28s 2ms/step - loss: 1395.4691 - mae: 1395.4691
Epoch 9/10
12820/12820 [==============================] - 31s 2ms/step - loss: 1379.0150 - mae: 1379.0150
Epoch 10/10
12820/12820 [=============

In [124]:
y_pred = prod_mass_flow_model.predict(X_test_normal)

3205/3205 [==============================] - 6s 2ms/step


In [125]:
y_pred

array([[509073.38 , 194866.1  ],
       [188156.56 ,  55655.934],
       [513431.72 , 185600.34 ],
       ...,
       [ 50163.203, 192702.7  ],
       [290650.7  ,  36081.438],
       [702582.5  , 249903.23 ]], dtype=float32)

In [126]:
y_test

,"rich_amine mass flow, kg/h","sweet_gas mass flow, kg/h"
337566,507815.414547,196179.585453
201781,187562.842578,56036.157422
215224,515275.821637,183408.178363
8834,283970.774396,148700.225604
394112,254584.050699,38068.949301
...,...,...
396710,176328.112169,255169.887831
447239,704293.420295,126707.579705
217230,49227.100639,193539.899361
397167,288826.341559,37772.658441


In [121]:
lean_amine_mass_flow_model.save('./models/amine_treatment/amine_treatment_lean_amine_mass_flow.h5')

## Модель для определения молярного расхода H2S и CO2 в продуктовых потоках

In [19]:
mini_data = data.sample(frac=0.95)
len(mini_data)

487160

In [20]:
sulphur = mini_data[[
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
    'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac',
    'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
    'feed_gas MDEAmine mol frac', 'feed_gas molar flow, kgmol/h', 'feed_gas H2S molar flow, kgmol/h',
    'feed_gas CO2 molar flow, kgmol/h',
    'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
    'lean_amine molar flow, kgmol/h', 'lean_amine H2S molar flow, kgmol/h', 'lean_amine CO2 molar flow, kgmol/h',
    'rich_amine temperature, C', 'rich_amine mass flow, kg/h', 'rich_amine molecular weight', 'rich_amine H2S mol frac',
    'rich_amine CO2 mol frac',
    'rich_amine molar flow, kgmol/h', 'rich_amine H2S molar flow, kgmol/h', 'rich_amine CO2 molar flow, kgmol/h'
]]

In [21]:
X = sulphur[[
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
    'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac',
    'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
    'feed_gas MDEAmine mol frac', 'feed_gas molar flow, kgmol/h', 'feed_gas H2S molar flow, kgmol/h',
    'feed_gas CO2 molar flow, kgmol/h',
    'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
    'lean_amine molar flow, kgmol/h', 'lean_amine H2S molar flow, kgmol/h', 'lean_amine CO2 molar flow, kgmol/h',
    'rich_amine temperature, C', 'rich_amine mass flow, kg/h', 'rich_amine molecular weight', 'rich_amine H2S mol frac',
    'rich_amine CO2 mol frac',
    'rich_amine molar flow, kgmol/h'
]]

y = sulphur[['rich_amine H2S molar flow, kgmol/h', 'rich_amine CO2 molar flow, kgmol/h']]

# Собираем трансформер столбцов
ct = make_column_transformer(
    (MinMaxScaler(), [
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
    'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac',
    'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
    'feed_gas MDEAmine mol frac', 'feed_gas molar flow, kgmol/h', 'feed_gas H2S molar flow, kgmol/h',
    'feed_gas CO2 molar flow, kgmol/h',
    'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
    'lean_amine molar flow, kgmol/h', 'lean_amine H2S molar flow, kgmol/h', 'lean_amine CO2 molar flow, kgmol/h',
    'rich_amine temperature, C', 'rich_amine mass flow, kg/h', 'rich_amine molecular weight', 'rich_amine H2S mol frac',
    'rich_amine CO2 mol frac',
    'rich_amine molar flow, kgmol/h'
    ])
)

# Создаем обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Загружаем тренировочную выборку
ct.fit(X_train)

# Нормализуем данные
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

X_train_normal.shape, y_train.shape

((389728, 38), (389728, 2))

In [28]:
# Set random set
tf.random.set_seed(42)

# 1. Create a model
prod_sour_comp_molar_flow_model = tf.keras.Sequential([
    tf.keras.layers.Dense(38, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(2)
])

# 2. Compile the model
prod_sour_comp_molar_flow_model.compile(
    loss='mae',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['mae']
)

# 3. Fit the model
prod_sour_comp_molar_flow_model.fit(X_train_normal, y_train, epochs=50)

Epoch 1/50
12179/12179 [==============================] - 28s 2ms/step - loss: 9.7967 - mae: 9.7967
Epoch 2/50
12179/12179 [==============================] - 27s 2ms/step - loss: 3.3948 - mae: 3.3948
Epoch 3/50
12179/12179 [==============================] - 28s 2ms/step - loss: 2.4351 - mae: 2.4351
Epoch 4/50
12179/12179 [==============================] - 26s 2ms/step - loss: 1.9254 - mae: 1.9254
Epoch 5/50
12179/12179 [==============================] - 28s 2ms/step - loss: 1.7543 - mae: 1.7543
Epoch 6/50
12179/12179 [==============================] - 25s 2ms/step - loss: 1.6269 - mae: 1.6269
Epoch 7/50
12179/12179 [==============================] - 27s 2ms/step - loss: 1.5322 - mae: 1.5322
Epoch 8/50
12179/12179 [==============================] - 27s 2ms/step - loss: 1.4521 - mae: 1.4521
Epoch 9/50
12179/12179 [==============================] - 26s 2ms/step - loss: 1.4221 - mae: 1.4221
Epoch 10/50
12179/12179 [==============================] - 32s 3ms/step - loss: 1.3372 - mae: 1.3372

In [29]:
y_pred = prod_sour_comp_molar_flow_model.predict(X_test_normal)

3045/3045 [==============================] - 5s 2ms/step


In [36]:
x = 213
pred = y_pred[x]
pred

array([77.347664, 43.99638 ], dtype=float32)

In [37]:
test = y_test.iloc[x]
test

rich_amine H2S molar flow, kgmol/h    77.979954
rich_amine CO2 molar flow, kgmol/h    44.592127
Name: 506935, dtype: float64

In [10]:
x = 7766
pred = y_pred[x][0].round(4)
pred

18.9377

In [11]:
test = round(y_test['rich_amine H2S molar flow, kgmol/h'].tolist()[x], 4)
test

18.7743

In [12]:
100 - (pred * 100) / test

-0.8703401543942846

In [13]:
X_test['feed_gas H2S molar flow, kgmol/h']

202762    124.096622
361145    184.753928
75864     305.992477
81049     125.205983
36368     222.311758
             ...    
422391     38.369554
406940    341.794414
511212    424.914242
435058     28.545601
361942    347.108731
Name: feed_gas H2S molar flow, kgmol/h, Length: 87176, dtype: float64

In [14]:
y_test

,"rich_amine H2S molar flow, kgmol/h"
202762,125.793648
361145,188.287139
75864,309.937196
81049,128.445327
36368,227.467310
...,...
422391,44.014625
406940,344.425770
511212,426.669771
435058,29.845525


In [19]:
initial_molar_flow = X_test['feed_gas H2S molar flow, kgmol/h'] + X_test['lean_amine H2S molar flow, kgmol/h']
sweet_gas_h2s_molar_flow = initial_molar_flow - y_test['rich_amine H2S molar flow, kgmol/h']
sweet_gas_h2s_molar_flow

202762    0.031718
361145    0.175669
75864     0.087509
81049     0.154056
36368     0.052821
            ...   
422391    0.037613
406940    0.018678
511212    0.025162
435058    0.161222
361942    0.227482
Length: 87176, dtype: float64

In [38]:
prod_sour_comp_molar_flow_model.save('./models/amine_treatment/amine_treatment_prod_sour_comp_molar_flow_model.h5')

## Модель для определения молекулярного веса сырья и продуктов

In [21]:
mini_data = data.sample(frac=0.85)
len(mini_data)

435880

In [33]:
molar_weight = mini_data[[
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
    'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac',
    'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
    'feed_gas MDEAmine mol frac', 'feed_gas molar flow, kgmol/h', 'feed_gas molecular weight',
    'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
    'lean_amine molar flow, kgmol/h', 'lean_amine molecular weight',
    'rich_amine temperature, C', 'rich_amine mass flow, kg/h', 'rich_amine molecular weight',
    'rich_amine molar flow, kgmol/h', 'sweet_gas molecular weight'
]]

X = molar_weight[[
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
    'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac',
    'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
    'feed_gas MDEAmine mol frac', 'feed_gas molar flow, kgmol/h', 
    'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
    'lean_amine molar flow, kgmol/h', 'lean_amine molecular weight',
    'rich_amine temperature, C', 'rich_amine mass flow, kg/h', 'rich_amine molecular weight',
    'rich_amine molar flow, kgmol/h'
]]

y = molar_weight[[
    'feed_gas molecular weight', 'lean_amine molecular weight', 'rich_amine molecular weight', 'sweet_gas molecular weight'
]]

# Собираем трансформер столбцов
ct = make_column_transformer(
    (MinMaxScaler(), [
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
    'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac',
    'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
    'feed_gas MDEAmine mol frac', 'feed_gas molar flow, kgmol/h', 
    'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
    'lean_amine molar flow, kgmol/h', 'lean_amine molecular weight',
    'rich_amine temperature, C', 'rich_amine mass flow, kg/h', 'rich_amine molecular weight',
    'rich_amine molar flow, kgmol/h'
    ])
)

# Создаем обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Загружаем тренировочную выборку
ct.fit(X_train)

# Нормализуем данные
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

X_train_normal.shape, y_train.shape

((348704, 33), (348704, 4))

In [34]:
# Set random set
tf.random.set_seed(42)

# 1. Create a model
molar_weight_model = tf.keras.Sequential([
    tf.keras.layers.Dense(33, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(4)
])

# 2. Compile the model
molar_weight_model.compile(
    loss='mae',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['mae']
)

# 3. Fit the model
molar_weight_model.fit(X_train_normal, y_train, epochs=25)

Epoch 1/25
10897/10897 [==============================] - 25s 2ms/step - loss: 0.3204 - mae: 0.3204
Epoch 2/25
10897/10897 [==============================] - 24s 2ms/step - loss: 0.1415 - mae: 0.1415
Epoch 3/25
10897/10897 [==============================] - 23s 2ms/step - loss: 0.1286 - mae: 0.1286
Epoch 4/25
10897/10897 [==============================] - 24s 2ms/step - loss: 0.1167 - mae: 0.1167
Epoch 5/25
10897/10897 [==============================] - 23s 2ms/step - loss: 0.1097 - mae: 0.1097
Epoch 6/25
10897/10897 [==============================] - 23s 2ms/step - loss: 0.1045 - mae: 0.1045
Epoch 7/25
10897/10897 [==============================] - 24s 2ms/step - loss: 0.1004 - mae: 0.1004
Epoch 8/25
10897/10897 [==============================] - 23s 2ms/step - loss: 0.0948 - mae: 0.0948
Epoch 9/25
10897/10897 [==============================] - 24s 2ms/step - loss: 0.0913 - mae: 0.0913
Epoch 10/25
10897/10897 [==============================] - 23s 2ms/step - loss: 0.0891 - mae: 0.0891

In [35]:
y_pred = molar_weight_model.predict(X_test_normal)

2725/2725 [==============================] - 4s 1ms/step


In [46]:
x = 3286
pred = y_pred[x]
pred

array([26.512918, 37.67699 , 37.715523, 26.006319], dtype=float32)

In [47]:
test = y_test.iloc[x]
test

feed_gas molecular weight      26.595999
lean_amine molecular weight    37.699229
rich_amine molecular weight    37.788213
sweet_gas molecular weight     25.973943
Name: 306893, dtype: float64

In [45]:
100 - (25.267178 * 100) / 25.478810

0.8306196403992061

In [48]:
molar_weight_model.save('./models/amine_treatment/amine_treatment_molar_weight_model.h5')

## Модель для определения состава насыщенного амина

In [19]:
mini_data = data.sample(frac=0.85)
len(mini_data)

435880

In [20]:
list(mini_data.columns)

['feed_gas temperature, C',
 'feed_gas pressure, kPa',
 'feed_gas mass flow, kg/h',
 'feed_gas molecular weight',
 'feed_gas Mass density, kg/m3',
 'feed_gas actual liquid flow, m3/h',
 'feed_gas actual vapour flow, m3/h',
 'feed_gas CO2 mol frac',
 'feed_gas Methane mol frac',
 'feed_gas Ethane mol frac',
 'feed_gas Propane mol frac',
 'feed_gas i-Butane mol frac',
 'feed_gas n-Butane mol frac',
 'feed_gas i-Pentane mol frac',
 'feed_gas n-Pentane mol frac',
 'feed_gas H2S mol frac',
 'feed_gas H2O mol frac',
 'feed_gas MDEAmine mol frac',
 'feed_gas H2S ppm',
 'feed_gas CO2 ppm',
 'Absorber number of stages',
 'lean_amine temperature, C',
 'lean_amine pressure, kPa',
 'lean_amine mass flow, kg/h',
 'lean_amine molecular weight',
 'lean_amine Mass density, kg/m3',
 'lean_amine actual liquid flow, m3/h',
 'lean_amine actual vapour flow, m3/h',
 'lean_amine CO2 mol frac',
 'lean_amine Methane mol frac',
 'lean_amine Ethane mol frac',
 'lean_amine Propane mol frac',
 'lean_amine i-Butane

In [25]:
molar_flow = mini_data[[
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas molecular weight', 'feed_gas CO2 mol frac',
    'feed_gas Methane mol frac', 'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac',
    'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac',
    'feed_gas H2O mol frac', 'feed_gas MDEAmine mol frac', 'feed_gas molar flow, kgmol/h',
    'feed_gas H2S molar flow, kgmol/h', 'feed_gas CO2 molar flow, kgmol/h', 'feed_gas H2O molar flow, kgmol/h',
    'feed_gas MDEAmine molar flow, kgmol/h', 'lean_amine temperature, C', 'lean_amine mass flow, kg/h',
    'lean_amine molecular weight', 'lean_amine CO2 mol frac', 'lean_amine Methane mol frac', 'lean_amine Ethane mol frac',
    'lean_amine Propane mol frac', 'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac',
    'lean_amine i-Pentane mol frac', 'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac',
    'lean_amine MDEAmine mol frac', 'lean_amine molar flow, kgmol/h', 'lean_amine H2S molar flow, kgmol/h',
    'lean_amine CO2 molar flow, kgmol/h', 'lean_amine H2O molar flow, kgmol/h', 'lean_amine MDEAmine molar flow, kgmol/h',
    'rich_amine temperature, C', 'rich_amine mass flow, kg/h', 'rich_amine molecular weight',
    'rich_amine molar flow, kgmol/h', 'rich_amine H2O molar flow, kgmol/h', 'rich_amine MDEAmine molar flow, kgmol/h'
    ]]

X = molar_flow[[
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas molecular weight', 'feed_gas CO2 mol frac',
    'feed_gas Methane mol frac', 'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac',
    'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac',
    'feed_gas H2O mol frac', 'feed_gas MDEAmine mol frac', 'feed_gas molar flow, kgmol/h',
    'feed_gas H2S molar flow, kgmol/h', 'feed_gas CO2 molar flow, kgmol/h', 'feed_gas H2O molar flow, kgmol/h',
    'feed_gas MDEAmine molar flow, kgmol/h', 'lean_amine temperature, C', 'lean_amine mass flow, kg/h',
    'lean_amine molecular weight', 'lean_amine CO2 mol frac', 'lean_amine Methane mol frac', 'lean_amine Ethane mol frac',
    'lean_amine Propane mol frac', 'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac',
    'lean_amine i-Pentane mol frac', 'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac',
    'lean_amine MDEAmine mol frac', 'lean_amine molar flow, kgmol/h', 'lean_amine H2S molar flow, kgmol/h',
    'lean_amine CO2 molar flow, kgmol/h', 'lean_amine H2O molar flow, kgmol/h', 'lean_amine MDEAmine molar flow, kgmol/h',
    'rich_amine temperature, C', 'rich_amine mass flow, kg/h', 'rich_amine molecular weight',
    'rich_amine molar flow, kgmol/h',
]]

y = molar_flow[[
    'rich_amine H2O molar flow, kgmol/h', 'rich_amine MDEAmine molar flow, kgmol/h'
]]

# Собираем трансформер столбцов
ct = make_column_transformer(
    (MinMaxScaler(), [
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas molecular weight', 'feed_gas CO2 mol frac',
    'feed_gas Methane mol frac', 'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac',
    'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac',
    'feed_gas H2O mol frac', 'feed_gas MDEAmine mol frac', 'feed_gas molar flow, kgmol/h',
    'feed_gas H2S molar flow, kgmol/h', 'feed_gas CO2 molar flow, kgmol/h', 'feed_gas H2O molar flow, kgmol/h',
    'feed_gas MDEAmine molar flow, kgmol/h', 'lean_amine temperature, C', 'lean_amine mass flow, kg/h',
    'lean_amine molecular weight', 'lean_amine CO2 mol frac', 'lean_amine Methane mol frac', 'lean_amine Ethane mol frac',
    'lean_amine Propane mol frac', 'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac',
    'lean_amine i-Pentane mol frac', 'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac',
    'lean_amine MDEAmine mol frac', 'lean_amine molar flow, kgmol/h', 'lean_amine H2S molar flow, kgmol/h',
    'lean_amine CO2 molar flow, kgmol/h', 'lean_amine H2O molar flow, kgmol/h', 'lean_amine MDEAmine molar flow, kgmol/h',
    'rich_amine temperature, C', 'rich_amine mass flow, kg/h', 'rich_amine molecular weight',
    'rich_amine molar flow, kgmol/h',
    ])
)

# Создаем обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Загружаем тренировочную выборку
ct.fit(X_train)

# Нормализуем данные
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

X_train_normal.shape, y_train.shape

((348704, 42), (348704, 2))

In [32]:
# Set random set
tf.random.set_seed(42)

# 1. Create a model
rich_amine_MDEA_H2O_model = tf.keras.Sequential([
    tf.keras.layers.Dense(42, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(2)
])

# 2. Compile the model
rich_amine_MDEA_H2O_model.compile(
    loss='mae',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['mae']
)

# 3. Fit the model
rich_amine_MDEA_H2O_model.fit(X_train_normal, y_train, epochs=10)

Epoch 1/10
10897/10897 [==============================] - 26s 2ms/step - loss: 228.5014 - mae: 228.5014
Epoch 2/10
10897/10897 [==============================] - 24s 2ms/step - loss: 20.9134 - mae: 20.9134
Epoch 3/10
10897/10897 [==============================] - 22s 2ms/step - loss: 18.7550 - mae: 18.7550
Epoch 4/10
10897/10897 [==============================] - 24s 2ms/step - loss: 18.2388 - mae: 18.2388
Epoch 5/10
10897/10897 [==============================] - 24s 2ms/step - loss: 17.5759 - mae: 17.5759
Epoch 6/10
10897/10897 [==============================] - 23s 2ms/step - loss: 16.9229 - mae: 16.9229
Epoch 7/10
10897/10897 [==============================] - 23s 2ms/step - loss: 16.6913 - mae: 16.6913
Epoch 8/10
10897/10897 [==============================] - 22s 2ms/step - loss: 16.5228 - mae: 16.5228
Epoch 9/10
10897/10897 [==============================] - 23s 2ms/step - loss: 16.3314 - mae: 16.3314
Epoch 10/10
10897/10897 [==============================] - 23s 2ms/step - loss: 

In [33]:
y_pred = rich_amine_MDEA_H2O_model.predict(X_test_normal)

2725/2725 [==============================] - 5s 2ms/step


In [40]:
x = 321
y_pred[x]

array([4424.8296 ,  249.52454], dtype=float32)

In [41]:
y_test.iloc[x]

rich_amine H2O molar flow, kgmol/h         4413.055614
rich_amine MDEAmine molar flow, kgmol/h     253.193071
Name: 383727, dtype: float64